In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\JUAN\Documents\GitHub\TuneMyMood


In [156]:
df=pd.read_csv("spotifyclassification/data.csv")
df_300=df.head(301)
df_train=pd.read_csv("trained.csv")

df_train['study'].values.tolist()
df_300.dtypes

df_train2=pd.read_csv("trained2.csv")

In [158]:
df_train

,Unnamed: 0,Unnamed: 0.1,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,study
0,0,0,0.010200,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future,0
1,1,1,0.199000,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino,1
2,2,2,0.034400,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future,0
3,3,3,0.604000,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House,1
4,4,4,0.180000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys,2
5,5,5,0.004790,0.804,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4.0,0.264,1,Sneakin’,Drake,1
6,6,6,0.014500,0.739,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4.0,0.308,1,Childs Play,Drake,2
7,7,7,0.020200,0.266,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4.0,0.393,1,Gyöngyhajú lány,Omega,1
8,8,8,0.048100,0.603,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4.0,0.398,1,I've Seen Footage,Death Grips,0
9,9,9,0.002080,0.836,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4.0,0.386,1,Digital Animal,Honey Claws,0


In [5]:
# Organizando dados para teste. Todo Speechiness maior que 0.1 será considerado Study=0, menor será Study=1, indicandonque musicas
# com pouco vocal são boas para estudar

df_300['study'] = len(df_300)*[0]
for i in range(150):
    if df_300['speechiness'][i] > 0.1:
        
            df_300['study'][i] = 0
        
    else:
       
            df_300['study'][i] = 1
        
    

C:\Users\JUAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\JUAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\JUAN\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\JUAN\Anaconda3\lib\site-p

In [6]:
from sklearn.naive_bayes import MultinomialNB

#Valores que serão dados de cada musica(Vocal e Energia)
X = []
#Valores que serão entregues de 1(Musica boa para estudo) e 0(Musica ruim para estudo)
train_ans = []

for i in range(31):
    X.append([df_300['speechiness'][i],df_300['energy'][i]])
    train_ans.append(df_300['study'][i])
    
songs= MultinomialNB()

y = np.array(train_ans)
songs.fit(X=X,y=y)

#Usando o predict podemos tentar prever 5 valores de X
print(songs.predict(X[0:6]))

#Conferindo o predict com os valores reais
print(y[0:6])


#É preciso denotar que por terem sido usados apenas 30 valores para a calibragem, o fit nao ficará preciso.

[0 1 1 1 1 1]
[0 1 0 1 1 0]


In [53]:
df=df.iloc[:,:14] #retirando o nome da cançao e do artista
df["loudness"]*=-1 #transformando o atributo loudness em positivo

#df_train["study"].tolist() #passando o dataframe para uma lista

In [104]:
np.array(df[:31].values.tolist()).astype(np.float64)[0]

array([  0.00000000e+00,   1.02000000e-02,   8.33000000e-01,
         2.04600000e+05,   4.34000000e-01,   2.19000000e-02,
         2.00000000e+00,   1.65000000e-01,   8.79500000e+00,
         1.00000000e+00,   4.31000000e-01,   1.50062000e+02,
         4.00000000e+00,   2.86000000e-01])

In [134]:

train30_array=np.array(df[:31].values.tolist()).astype(np.float64) #passando o dataframe para uma lista com valores de type=float
train30_chosen=df_train["study"].tolist() #passando a as musicas classificadas (base de treino)

t2_array=np.array(df[29:57].values.tolist()).astype(np.float64) #passando o dataframe para uma lista com valores de type=float
t2_chosen=df_train[:28]["study"].tolist() #passando a as musicas classificadas (base de treino)



train30=MultinomialNB()
train30.fit(train30_array,train30_chosen) #fit do database de treino com as musicas classificadas

study_songs=[]
non_study_songs=[]
maybe_songs=[]

results=[]
for i in range(30): #algoritmo fazendo sua classificação de musicas novas
    results.append(train30.predict(df[i+30:i+31]))
    if train30.predict(df[i+30:i+31])==2:
        study_songs.append(df_300["song_title"][i+30:i+31])
    elif train30.predict(df[i+30:i+31])==1:
        maybe_songs.append(df_300["song_title"][i+30:i+31])
    else:
        non_study_songs.append(df_300["song_title"][i+30:i+31])
study_songs

train30.predict(df[29:57].values.tolist())


[[29.0,
  0.282,
  0.735,
  186096.0,
  0.4,
  0.125,
  1.0,
  0.0811,
  12.765999999999998,
  1.0,
  0.102,
  129.004,
  4.0,
  0.326],
 [30.0,
  0.007909999999999999,
  0.45,
  220934.0,
  0.449,
  0.0,
  2.0,
  0.17300000000000001,
  12.037,
  1.0,
  0.156,
  135.45,
  4.0,
  0.13699999999999998],
 [31.0,
  0.0219,
  0.897,
  285240.0,
  0.642,
  1.31e-06,
  2.0,
  0.159,
  8.828,
  1.0,
  0.0534,
  127.984,
  4.0,
  0.27],
 [32.0,
  0.0358,
  0.9590000000000001,
  213000.0,
  0.598,
  0.0,
  8.0,
  0.358,
  5.534,
  1.0,
  0.0713,
  127.029,
  4.0,
  0.424],
 [33.0,
  0.0324,
  0.742,
  280831.0,
  0.5589999999999999,
  0.0,
  5.0,
  0.0987,
  5.876,
  1.0,
  0.141,
  146.923,
  4.0,
  0.429],
 [34.0,
  0.11800000000000001,
  0.805,
  232880.0,
  0.687,
  0.0,
  9.0,
  0.0842,
  7.771,
  1.0,
  0.0458,
  119.999,
  4.0,
  0.6829999999999999],
 [35.0,
  0.0428,
  0.667,
  230693.0,
  0.581,
  0.0,
  0.0,
  0.183,
  5.164,
  0.0,
  0.32299999999999995,
  110.95299999999999,
  4.0,
  

In [10]:
non_study_songs

[30    Milly Rock
 Name: song_title, dtype: object, 32    Best Friend
 Name: song_title, dtype: object, 33    Hercules
 Name: song_title, dtype: object, 34    Who Be Lovin Me (feat. ILOVEMAKONNEN)
 Name: song_title, dtype: object, 35    Check
 Name: song_title, dtype: object, 36    Poppin' My Collar
 Name: song_title, dtype: object, 37    Jumpman
 Name: song_title, dtype: object, 38    7/11
 Name: song_title, dtype: object, 39    Somebody's Watching Me
 Name: song_title, dtype: object, 41    I Don't Sell Molly No More
 Name: song_title, dtype: object, 42    Versace Python
 Name: song_title, dtype: object, 44    OOOUUU
 Name: song_title, dtype: object, 45    Return Of The Mack - C & J Street Mix
 Name: song_title, dtype: object, 47    Right Place Wrong Time
 Name: song_title, dtype: object, 48    Azon de ma gnin kpevi
 Name: song_title, dtype: object, 50    Annie
 Name: song_title, dtype: object, 51    The Chase
 Name: song_title, dtype: object, 52    Coming Home
 Name: song_title, dtyp

[array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([2])]

In [11]:
maybe_songs

[31    Blasé (feat. Future & Rae Sremmurd)
 Name: song_title, dtype: object, 40    Bang That
 Name: song_title, dtype: object]

In [12]:
#print(df_train2["study"].tolist())
np.array(results).shape

(30, 1)

In [13]:
#df_train2["study"].values.reshape(1,-1).shape
len(t2_array)

28

In [14]:
train30.score(t2_array,t2_chosen) #teste dos resultados

0.42857142857142855

In [15]:
# print(np.array(results))
# formated=[]
# for i in range(len(results)):
#     formated.append(results[i][0])

# print(formated)
# print((df_train2["study"].values.tolist()))
# train30.score(X=np.array(results),y=(df_train2["study"].values.tolist()))
# #results

In [120]:
df2=pd.read_csv("spotifyclassification/data.csv")
# a=list(range(15))
# a.append(16)
#df2=df2.iloc[:,1:14]
df2.head(10)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286
1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588
2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173
3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230
4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904
5,0.00479,0.804,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4.0,0.264
6,0.01450,0.739,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4.0,0.308
7,0.02020,0.266,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4.0,0.393
8,0.04810,0.603,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4.0,0.398
9,0.00208,0.836,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4.0,0.386


In [124]:
df2.iloc[:,1:14].to_dict('records')

[{'danceability': 0.83299999999999996,
  'duration_ms': 204600.0,
  'energy': 0.434,
  'instrumentalness': 0.021899999999999999,
  'key': 2.0,
  'liveness': 0.16500000000000001,
  'loudness': -8.7949999999999999,
  'mode': 1.0,
  'speechiness': 0.43099999999999999,
  'tempo': 150.06200000000001,
  'time_signature': 4.0,
  'valence': 0.28600000000000003},
 {'danceability': 0.74299999999999999,
  'duration_ms': 326933.0,
  'energy': 0.35899999999999999,
  'instrumentalness': 0.00611,
  'key': 1.0,
  'liveness': 0.13699999999999998,
  'loudness': -10.401,
  'mode': 1.0,
  'speechiness': 0.079399999999999998,
  'tempo': 160.083,
  'time_signature': 4.0,
  'valence': 0.58799999999999997},
 {'danceability': 0.83799999999999997,
  'duration_ms': 185707.0,
  'energy': 0.41200000000000003,
  'instrumentalness': 0.000234,
  'key': 2.0,
  'liveness': 0.159,
  'loudness': -7.1479999999999997,
  'mode': 1.0,
  'speechiness': 0.28899999999999998,
  'tempo': 75.043999999999997,
  'time_signature': 4.

In [148]:
from sklearn.feature_extraction import DictVectorizer
vec=DictVectorizer()
df2["loudness"]*=-1
treinamento=vec.fit_transform(df2[:31].to_dict('records')).toarray()

In [151]:
treinamento[0]

array([  1.02000000e-02,   8.33000000e-01,   2.04600000e+05,
         4.34000000e-01,   2.19000000e-02,   2.00000000e+00,
         1.65000000e-01,   8.79500000e+00,   1.00000000e+00,
         4.31000000e-01,   1.50062000e+02,   4.00000000e+00,
         2.86000000e-01])

13

In [155]:
oi=MultinomialNB()

oi.fit(treinamento,train30_chosen)
oi.predict(vec.fit_transform(df2[31:32].to_dict('records')).toarray())

array([1])